In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time

In [21]:
tf.executing_eagerly()

True

In [4]:
data = pd.read_csv('arxiv_data.csv')
data = data.drop(columns=['summaries','terms'])
data = data.sample(frac=1)
data = data[0:500]
data.head()

,titles
24306,Structured Query-Based Image Retrieval Using S...
15142,Intelligent Icing Detection Model of Wind Turb...
40637,CORE: Color Regression for Multiple Colors Fas...
41320,"Fast, Accurate, and Simple Models for Tabular ..."
28183,Approach for modeling single branches of meado...


In [5]:
data.shape

(500, 1)

In [ ]:
'''
data2 = pd.read_csv('arxiv_data_210930-054931.csv')
data2 = data2.drop(columns=['abstracts','terms'])
data2 = data2.sample(frac=1)
#data2 = data2[0:500]
data2.head()
'''

In [ ]:
#data2.shape

In [ ]:
#data2.head()

In [ ]:
#data.shape

In [6]:
terms = data.titles.tolist()
text = ''
for t in terms:
    text=text+' ' +t
text = text[1::]

In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

72 unique characters


In [8]:
text_as_int = np.array([char2idx[c] for c in text])

In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [10]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
# Batch size
BATCH_SIZE = 10

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((10, 100), (10, 100)), types: (tf.int32, tf.int32)>

In [12]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 100

# Number of RNN units
rnn_units = 100

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [14]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (10, None, 100)           7200      
_________________________________________________________________
gru (GRU)                    (10, None, 100)           60600     
_________________________________________________________________
dense (Dense)                (10, None, 72)            7272      
Total params: 75,072
Trainable params: 75,072
Non-trainable params: 0
_________________________________________________________________


In [15]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [16]:
model.compile(optimizer='adam', loss=loss,run_eagerly=True)

In [17]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [18]:
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
36/36 [==============================] - 12s 314ms/step - loss: 3.7705
Epoch 2/50
36/36 [==============================] - 12s 316ms/step - loss: 3.1354
Epoch 3/50
36/36 [==============================] - 12s 323ms/step - loss: 2.8128
Epoch 4/50
36/36 [==============================] - 11s 316ms/step - loss: 2.5694
Epoch 5/50
36/36 [==============================] - 12s 319ms/step - loss: 2.4365
Epoch 6/50
36/36 [==============================] - 12s 324ms/step - loss: 2.3344
Epoch 7/50
36/36 [==============================] - 11s 313ms/step - loss: 2.2490
Epoch 8/50
36/36 [==============================] - 12s 337ms/step - loss: 2.1673
Epoch 9/50
36/36 [==============================] - 12s 325ms/step - loss: 2.0958
Epoch 10/50
36/36 [==============================] - 12s 327ms/step - loss: 2.0286
Epoch 11/50
36/36 [==============================] - 12s 322ms/step - loss: 1.9676
Epoch 12/50
36/36 [==============================] - 14s 395ms/step - loss: 1.9109
Epoch 13/50
3

In [19]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [20]:
def generate_text(model, start_string,t):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = t

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
t_01 = (generate_text(model, start_string=u"Transformer",t=0.1))
t_02 = (generate_text(model, start_string=u"Bio ",t=0.2))
t_06 = (generate_text(model, start_string=u"Neural ",t=0.6))
t_08 = (generate_text(model, start_string=u"Liquidation",t=0.8))
t_1 = (generate_text(model, start_string=u"Support ",t=1.0))

In [ ]:
T = [0.1,0.2,0.6,0.8,1.00]
strings = [t_01,t_02,t_06,t_08,t_1]
for i in range(len(T)):
    print('Temperature = %.1f \n Strings Produced = %s'%(T[i],strings[i]))

In [ ]:
import joblib
joblib.dump(model, 'topic_generate.pkl')